# Workgroup 4

- Janice Mamani

In [ ]:
using RData, LinearAlgebra, DataFrames, Statistics, Random, Distributions, DataStructures, NamedArrays, PrettyTables,
        Plots, StatsBase,StatsPlots
using DelimitedFiles, Lasso
using CategoricalArrays
using GLM
import CodecBzip2

# Bootstraping

In [ ]:
penn, head = readdlm("/content/penn_jae.dat", header=true, Float64)
penn =DataFrame(penn, vec(head))
penn


Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10824.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10635.0,2.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,10551.0,5.0,18.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10824.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,10747.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,10544.0,6.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,10845.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8,10670.0,3.0,3.0,3.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
9,10768.0,3.0,28.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Filter only rows which have tg = 0 or 4
penn = filter(row -> row[:tg] in [4,0], penn)

replace!(penn.tg, 4 => 1)
rename!(penn, "tg" => "T4")

# from float to string
penn[!,:dep] = string.(penn[!,:dep])

# dep varaible in categorical format
penn[!,:dep] = categorical(penn[!,:dep]);


In [ ]:
penn

Row,abdt,T4,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10824.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10824.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,10747.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,10607.0,1.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5,10831.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
6,10845.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
7,10831.0,0.0,9.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
8,10859.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,10516.0,0.0,15.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
function boot_fn(data,index)
            ols_1 = lm(@formula(log(inuidur1)~T4+ (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)), penn[index,:])
            T4 = GLM.coeftable(ols_1).cols[1][2]
            female = GLM.coeftable(ols_1).cols[1][3]
            black = GLM.coeftable(ols_1).cols[1][4]
            return [T4, female, black]
end

boot_fn (generic function with 1 method)

In [ ]:
function boot_2(data,func,R)
            T4_coef = []
            fem_coef = []
            black_coef = []
            for i in 1:R
                append!(T4_coef,func(data,sample([1:5099;], 5099, replace = true))[1])
                append!(fem_coef,func(data,sample([1:5099;], 5099, replace = true))[2])
                append!(black_coef,func(data,sample([1:5099;], 5099, replace = true))[3])
            end
        table = NamedArray(zeros(3, 3))

        table[1,2] = mean(T4_coef)
        table[1,3] = std(T4_coef, corrected=true)
        table[2,2] = mean(fem_coef)
        table[2,3] = std(fem_coef, corrected=true)
        table[3,2] = mean(black_coef)
        table[3,3] = std(black_coef, corrected=true)
        T = DataFrame(table, [ :"Variable", :"Coefficient (boostrap)", :"Standard error (boostrap)"])
        T[!,:Variable] = string.(T[!,:Variable])

        T[1,1] = "T4"
        T[2,1] = "Female"
        T[3,1] = "Black"

        bootstrap_statistics = Dict{String,Any}("Table" => T, "T4" => T4_coef, "Female" => fem_coef, "Black" => black_coef)
    return bootstrap_statistics
end


boot_2 (generic function with 1 method)

In [ ]:
lm(@formula(log(inuidur1)~T4+ (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)), penn[1:5099,:])


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(inuidur1)) ~ 1 + T4 + female + black + othrace + dep + q2 + q3 + q4 + q5 + q6 + agelt35 + agegt54 + durable + lusd + husd

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)   2.17846      0.159015   13.70    <1e-41   1.86672      2.4902
T4           -0.0716925    0.0354633  -2.02    0.0433  -0.141216    -0.00216922
female        0.126368     0.0348249   3.63    0.0003   0.0580965    0.19464
black        -0.293768     0.0529756  -5.55    <1e-07  -0.397623    -0.189913
othrace      -0.472445     0.198398   -2.38    0.0173  -0.86139     -0.0835005
dep: 1.0      0.0298669    0.0541402   0.55    0

In [ ]:
boot_2(penn,boot_fn,1000)["Table"]


Row,Variable,Coefficient (boostrap),Standard error (boostrap)
,String,Float64,Float64
1,T4,-0.071531,0.0350679
2,Female,0.125219,0.034079
3,Black,-0.29691,0.0589442
